imports

In [1]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm as tqdm
from tqdm.auto import trange
import numpy as np

import warnings
warnings.filterwarnings("ignore")
sys.path.insert(1, '../train')
sys.path.insert(1, '../misc')

import config_plots, TrialStatistics
import CNN, dataLoader
from configParser import ConfigParser, getModelName
config_plots.global_settings()

parameters

In [2]:
experimentName = "Easy"
experimentsPath="/raid/elhamod/Fish/official_experiments/" # Where experiment results will be produced
dataPath="/raid/elhamod/Fish/Curated4" # Where data is

generate_confusion_matrix = False

cuda=1

Cuda

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 1


Setup

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

experimentPathAndName = os.path.join(experimentsPath, experimentName)

# instantiate trial stat object
results_dir = os.path.join(experimentPathAndName, "results")
ts = TrialStatistics.TrialStatistics(results_dir)
ts_coarse = TrialStatistics.TrialStatistics(results_dir, "coarse")

datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  

Show and save trial statistics

In [5]:

with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)

        # For analyzing experiments, we don't care about augmentation
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        fineList = train_loader.dataset.csv_processor.getFineList()
        coarseList = train_loader.dataset.csv_processor.getCoarseList()
        architecture = {
            "fine": len(fineList),
            "coarse" : len(coarseList)
        }


        
        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            
            # Train/Load model
            print(CNN.getModelFile(trialName))
            model = CNN.create_model(architecture, experiment_params, device=cuda)
            if os.path.exists(CNN.getModelFile(trialName)):
                df, epochs, time_elapsed = CNN.loadModel(model, trialName, device=cuda)
                
                # Update trial outcomes for statistics
                predlist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params, device=cuda)
                loss = CNN.getCrossEntropy(predlist, lbllist)
                avg_prob = CNN.getAvgProbCorrectGuess(predlist, lbllist)
                topk = CNN.top_k_acc(predlist, lbllist, topk=(3,5))
                
                predlist, lbllist = CNN.getPredictions(predlist, lbllist)
                labels =list(map(lambda x: x[1] + " - " + str(x[0]), enumerate(fineList))) 
                ts.addTrialPredictions(experiment_params,i, predlist, lbllist, labels)
                micro_f1 = f1_score(lbllist.cpu(), predlist.cpu(), average='macro')

                predlist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params, 'coarse', device=cuda)
                predlist, lbllist = CNN.getPredictions(predlist, lbllist)
                labels =list(map(lambda x: x[1] + " - " + str(x[0]), enumerate(coarseList)))
                ts_coarse.addTrialPredictions(experiment_params,i, predlist, lbllist, labels)
                micro_f1_coarse = f1_score(lbllist.cpu(), predlist.cpu(), average='macro')

                predlist, lbllist = CNN.getLoaderPredictionProbabilities(validation_loader, model, experiment_params, device=cuda)
                predlist, lbllist = CNN.getPredictions(predlist, lbllist)
                macro_f1_val = f1_score(lbllist.cpu(), predlist.cpu(), average='macro')

                score = {'loss': loss,
                         'average correct guess prob': avg_prob,
                         'macro f1 test fine': micro_f1,
                         'macro f1 test coarse': micro_f1_coarse,
                         'macro f1 validation fine': macro_f1_val,
                         'time': time_elapsed,
                         'epochs': epochs,
                         'top-3': topk[0].cpu().numpy(),
                         'top-5': topk[1].cpu().numpy(),
                        }

                ts.addTrial(experiment_params,
                    score, i)
            else:
                print("Model {0} not found!".format(trialName))
        
        bar.update()

{'image_path': 'Easy_100', 'img_res': 448, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.0001, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'HGNN', 'lambda': 0.01, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.01, 'adaptive_alpha': 0.9, 'pretrained': True}
Creating datasets...
Creating datasets... Done.
Creating loaders...
Creating loaders... Done.


/raid/elhamod/Fish/official_experiments/Easy/models/722617467486e177214160d664ac53cfc7705dfc4842c031a0d852d0/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254/trial_confusion_matrix_722617467486e177214160d664ac53cfc7705dfc4842c031a0d852d0.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254/coarse_trial_confusion_matrix_722617467486e177214160d664ac53cfc7705dfc4842c031a0d852d0.csv
/raid/elhamod/Fish/official_experiments/Easy/models/6686817b9cc7a39ff6acd6dab16859e287cb7b747eb9b5bb72239d91/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254/trial_confusion_matrix_6686817b9cc7a39ff6acd6dab16859e287cb7b747eb9b5bb72239d91.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254/coarse_trial_confusion_

/raid/elhamod/Fish/official_experiments/Easy/models/d01fa04100c92f6b8d6f0c80f64c9bfc354ceab094e191826caaf880/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876/trial_confusion_matrix_d01fa04100c92f6b8d6f0c80f64c9bfc354ceab094e191826caaf880.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876/coarse_trial_confusion_matrix_d01fa04100c92f6b8d6f0c80f64c9bfc354ceab094e191826caaf880.csv
/raid/elhamod/Fish/official_experiments/Easy/models/b3f66fec65fb80293f9f81ddf0aa8a4801f9e0360142b35239de7b8e/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876/trial_confusion_matrix_b3f66fec65fb80293f9f81ddf0aa8a4801f9e0360142b35239de7b8e.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876/coarse_trial_confusion_

/raid/elhamod/Fish/official_experiments/Easy/models/c6230130e44001a9b123b44431fdef39d32a25d1a928ec96676a4cb2/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/fbf4c275e857df725e65ca95f294905a0966e68874b376dc8d1d1da9/trial_confusion_matrix_c6230130e44001a9b123b44431fdef39d32a25d1a928ec96676a4cb2.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/fbf4c275e857df725e65ca95f294905a0966e68874b376dc8d1d1da9/coarse_trial_confusion_matrix_c6230130e44001a9b123b44431fdef39d32a25d1a928ec96676a4cb2.csv
/raid/elhamod/Fish/official_experiments/Easy/models/c8602b33e64e8c3edaf179fb3339690231ff0b781b127e9c75cdd761/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/fbf4c275e857df725e65ca95f294905a0966e68874b376dc8d1d1da9/trial_confusion_matrix_c8602b33e64e8c3edaf179fb3339690231ff0b781b127e9c75cdd761.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/fbf4c275e857df725e65ca95f294905a0966e68874b376dc8d1d1da9/coarse_trial_confusion_

/raid/elhamod/Fish/official_experiments/Easy/models/14f84fc022b7e7cb4f3330a0f735694d47999bed71128c6dac61fe11/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/db974121c4a931769e380ce11ce9774e60998237308ca1aeb2b96a99/trial_confusion_matrix_14f84fc022b7e7cb4f3330a0f735694d47999bed71128c6dac61fe11.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/db974121c4a931769e380ce11ce9774e60998237308ca1aeb2b96a99/coarse_trial_confusion_matrix_14f84fc022b7e7cb4f3330a0f735694d47999bed71128c6dac61fe11.csv
/raid/elhamod/Fish/official_experiments/Easy/models/88d35212f84ce39cadc0adda32e6a511c2041d7f168940698180ed0d/finalModel.pt
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/db974121c4a931769e380ce11ce9774e60998237308ca1aeb2b96a99/trial_confusion_matrix_88d35212f84ce39cadc0adda32e6a511c2041d7f168940698180ed0d.csv
Saving  /raid/elhamod/Fish/official_experiments/Easy/results/db974121c4a931769e380ce11ce9774e60998237308ca1aeb2b96a99/coarse_trial_confusion_

In [6]:
# Save experiment results
ts.saveStatistics()
ts.showStatistics()

In [7]:
ts.saveStatistics(False)
ts.showStatistics(False)

,image_path,img_res,augmented,batchSize,learning_rate,numOfTrials,fc_layers,modelType,lambda,tl_model,link_layer,adaptive_smoothing,adaptive_lambda,adaptive_alpha,pretrained,loss,average correct guess prob,macro f1 test fine,macro f1 test coarse,macro f1 validation fine,time,epochs,top-3,top-5,experimentHash,trial,trialHash
0,Easy_100,448,True,64,0.0001,5,1,HGNN,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.060238,0.611330,0.924904,0.964990,0.892526,7397.736515,31.0,98.406380,98.539177,4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254,0,722617467486e177214160d664ac53cfc7705dfc4842c031a0d852d0
1,Easy_100,448,True,64,0.0001,5,1,HGNN,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.066351,0.605071,0.919533,0.967750,0.893057,8846.029403,36.0,97.476761,98.539177,4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254,1,6686817b9cc7a39ff6acd6dab16859e287cb7b747eb9b5bb72239d91
2,Easy_100,448,True,64,0.0001,5,1,HGNN,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.064990,0.606551,0.901596,0.980435,0.893042,7872.952063,32.0,98.007973,98.804787,4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254,2,ad588cadfd61384b4e5e62d0f926c85636ba65eadea16ce5d51b789b
3,Easy_100,448,True,64,0.0001,5,1,HGNN,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.045503,0.626510,0.910369,0.957858,0.865014,10367.604944,42.0,97.476761,98.671982,4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254,3,b3ab8dbec807dff011941a71ab9d048ec45f6cdbf6a640b33c4bf3f2
4,Easy_100,448,True,64,0.0001,5,1,HGNN,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.045725,0.626202,0.919089,0.972296,0.911461,10381.803864,42.0,98.804787,99.468796,4a8412482c9058b4e7a6bac744f002520455703ce8fb46c7006df254,4,b4d477e8e08d2f67f226408c27255a1e73b8cb41e400f643acf90bba
5,Easy_100,448,True,64,0.0001,5,1,BB,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.027120,0.645418,0.931576,0.990886,0.902610,15625.164412,65.0,98.804787,99.203194,02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876,0,d01fa04100c92f6b8d6f0c80f64c9bfc354ceab094e191826caaf880
6,Easy_100,448,True,64,0.0001,5,1,BB,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.047051,0.624871,0.909313,0.981823,0.888147,9993.154941,41.0,97.742371,98.406380,02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876,1,b3f66fec65fb80293f9f81ddf0aa8a4801f9e0360142b35239de7b8e
7,Easy_100,448,True,64,0.0001,5,1,BB,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.028355,0.644206,0.921982,0.980939,0.904018,16283.314431,67.0,98.804787,99.468796,02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876,2,f47948fca32a46919ef2d1d9a4e8eb4e38e1bab29c2f20a1a96e2f37
8,Easy_100,448,True,64,0.0001,5,1,BB,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.028401,0.644232,0.918435,0.987628,0.894631,20005.397595,82.0,97.609566,98.273575,02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876,3,f6753a3c38fd694b092fd1e08322d6c676233430b038cf3d331935ec
9,Easy_100,448,True,64,0.0001,5,1,BB,0.01,ResNet18,avgpool,True,0.01,0.9,True,3.070850,0.600545,0.917895,0.965739,0.871306,6783.394021,28.0,98.140770,98.671982,02d1f000996d2ba32a22b23296f3daa2b1533cf81b050de33f016876,4,218f46b1f2dab1a9f1ada39e431a9ec9dd36c528b8cea96b708c46d4


Show and save confusion matrix

In [8]:
if generate_confusion_matrix:
    with tqdm(total=number_of_experiments, desc="experiment") as bar:
        for experiment_params in config_parser.getExperiments():
            print(experiment_params)

            ts.printTrialConfusionMatrix(experiment_params, fineList , printOutput=True)
            ts_coarse.printTrialConfusionMatrix(experiment_params,  coarseList, printOutput=True)
            
            datasetManager.updateParams(config_parser.fixPaths({**experiment_params,**{'augmented': False}}))
            train_loader, validation_loader, test_loader = datasetManager.getLoaders()
            ts.printF1table(experiment_params, test_loader.dataset)

            bar.update()

In [9]:
ts.df = ts.df.copy().replace(r"_", "", regex=True)
ts.df.columns = ts.df.columns.str.replace('_','')